# Q1. Install MLflow

In [1]:
# Q1
! mlflow --version

mlflow, version 2.7.1


# Q2. Download and preprocess the data

In [2]:
links = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet"
]

# download data to TAXI_DATA_FOLDER subdirectory replace if exists
for link in links:
    ! wget -N -P TAXI_DATA_FOLDER $link

--2024-05-24 09:40:15--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.161.3.141, 18.161.3.58, 18.161.3.119, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.161.3.141|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘TAXI_DATA_FOLDER/green_tripdata_2023-01.parquet’ not modified on server. Omitting download.

--2024-05-24 09:40:17--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.161.3.141, 18.161.3.58, 18.161.3.209, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.161.3.141|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘TAXI_DATA_FOLDER/green_tripdata_2023-02.parquet’ not modified on server. Omitting download.

--2024-05-24 09:40:18-- 

In [3]:
# download preprocess_data.py
! wget -N https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/preprocess_data.py

--2024-05-24 09:40:20--  https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/preprocess_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2510 (2.5K) [text/plain]
Saving to: ‘preprocess_data.py’

preprocess_data.py  100%[===================>]   2.45K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2024-05-24 09:40:21 (31.5 MB/s) - ‘preprocess_data.py’ saved [2510/2510]



In [4]:
! python preprocess_data.py --raw_data_path TAXI_DATA_FOLDER --dest_path ./output

In [5]:
# Q2
# number of files in output directory
import os
output_dir = './output'
num_files = len([name for name in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, name))])
num_files

4

# Q3. Train a model with autolog

In [7]:
# download train.py
# ! wget -N https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/train.py

In [6]:
# run train.py
! python train.py --data_path ./output

2024/05/24 09:40:38 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/24 09:40:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/envs/hw2-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
